<a href="https://colab.research.google.com/github/tankien76/CS114.K21/blob/master/Sarcasm_headlines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Danh sách nhóm

- Đoàn Xuân Minh - MSSV: 18521093
- Lê Trần Phúc Nguyên - MSSV: 18521157
- Phan Quang Tấn - MSSV: 18521377

# Mô tả bài toán
- Input: Các tiêu đề bài báo (headlines)
- Output: Tiêu đề châm biếm hay không châm biếm

Code tham khảo: https://towardsdatascience.com/sarcasm-detection-step-towards-sentiment-analysis-84cb013bb6db
                

Code tham khảo: https://pythonspot.com/nltk-stop-words/

# Prepare dataset

Thu thập các đường link của bài báo(article_link), tiêu đề(headlines) sarcasm(is_sarcastic=1) từ trang https://www.theonion.com và not sarcasm(is_sarcastic=0) từ trang https://www.huffingtonpost.com

Upload file kaggle.json để lấy dữ liệu từ kaggle

In [1]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


Tải dữ liệu về từ kaggle và giải nén để lấy dataset từ trang web kaggle (https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection)

In [2]:
!ls -lha kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 68 Jun 26 13:29 kaggle.json


In [3]:
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection

  0% 0.00/3.30M [00:00<?, ?B/s]
100% 3.30M/3.30M [00:00<00:00, 54.3MB/s]


In [4]:
!unzip news-headlines-dataset-for-sarcasm-detection.zip

Archive:  news-headlines-dataset-for-sarcasm-detection.zip
  inflating: Sarcasm_Headlines_Dataset.json  
  inflating: Sarcasm_Headlines_Dataset_v2.json  


# Feature engineering
Tiến hành xử lý dữ liệu(headline) :
- Loại bỏ các ký tự đặc biệt(dấu câu, số)
- Loại bỏ các stopword

download punkt trong thư viện nltk để tách từ khi xử lý dữ liệu

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

download stopwords trong thư viện nltk để xử lý các từ không cần thiết trong tiếng anh

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Import các thư viện sử dụng:
- pandas : đọc dữ liệu file json
- numpy : xử lý dữ liệu text
- nltk : xử lý ngôn ngữ tự nhiên 

In [7]:
import pandas as pd, numpy as np, re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

Đọc dữ liệu bằng thư viện pandas từ file Sarcasm_Headlines_Dataset_v2.json

In [8]:
# Loading data from json file
data = pd.read_json("Sarcasm_Headlines_Dataset_v2.json", lines = True)
data.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


Đưa dữ liệu headline và labels vào list tương ứng

In [9]:
features = data['headline']
labels = data['is_sarcastic']

Loại bỏ các ký tự đặc biệt, số và các từ không cần thiết trong headline để chuyển hóa thành vector vì .fit không thực hiện khi dữ liệu là string

In [10]:
features = features.apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))

In [11]:
stop_words = set(stopwords.words('english'))
for i in range (len(features)):
  word_tokens = word_tokenize(features[i])

  filtered_sentence = [w for w in word_tokens if not w in stop_words]

  filtered_sentence = []

  for w in word_tokens:
      if w not in stop_words:
          filtered_sentence.append(w)

Vector hóa dữ liệu văn bản bằng TF-IDF với max_feature = 5000 (dùng để tranh ngốn ram của máy):
- TF: tần suất xuất hiện của 1 từ trong 1 văn bản
- IDF: tần suất nghịch của 1 từ trong 1 tập văn bản
- .fit: Máy học từ vựng và tính toán idf( tần suất nghịch của 1 từ trong tập các văn bản) của train set.


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfIdfVec = TfidfVectorizer(max_features = 5000)
tfIdfVec.fit(features)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=5000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

Đưa dữ liệu headlines về matrix để training model

In [13]:
tf_idf_x = tfIdfVec.transform(features)

 Tiến hành chia dữ liệu train và test: theo tỉ lệ 90% training và 10% testing vì khi thực hiện theo tỉ lệ 80/20 hoặc 70/30 thì độ chính xác của các thuật toán giảm

In [14]:
from sklearn.model_selection import train_test_split
features_train, features_test, labels_train, labels_test = train_test_split(tf_idf_x.toarray(), labels, test_size = .1, random_state = 0)

# Model

Thêm thư viện sklearn gồm :
- Các model sử dụng : LinearSVC, Gaussian Naive Bayes, Logistic Regresstion, Random Forest
- train_test_split: để chia dữ liệu trong dataset thành train set vầ test set

In [15]:
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

Model Linear Support Vector Classifier

In [16]:
lsvc = LinearSVC()
lsvc.fit(features_train, labels_train)
print("train accuracy: ",lsvc.score(features_train, labels_train))
print("Test accuracy: ",lsvc.score(features_test, labels_test))

train accuracy:  0.9129945257599876
Test accuracy:  0.8424178895877009


Model Gaussian Naive Bayes

In [17]:
gnb = GaussianNB()
gnb.fit(features_train, labels_train)
print("train accuracy: ",gnb.score(features_train, labels_train))
print("Test accuracy: ",gnb.score(features_test, labels_test))

train accuracy:  0.8177971036999651
Test accuracy:  0.7578616352201258


Model Logistic Regression

In [18]:
lr = LogisticRegression()
lr.fit(features_train, labels_train)
print("train accuracy: ",lr.score(features_train, labels_train))
print("Test accuracy: ",lr.score(features_test, labels_test))

train accuracy:  0.8834103350545482
Test accuracy:  0.8452131376659678


Model Random Forest Classifier

In [19]:
rfc = RandomForestClassifier(n_estimators = 10, random_state = 0)
rfc.fit(features_train, labels_train)
print("train accuracy: ",rfc.score(features_train, labels_train))
print("Test accuracy: ",rfc.score(features_test, labels_test))

train accuracy:  0.9894397639476648
Test accuracy:  0.7917540181691125


=> Sau khi training và đánh giá model thông qua accuracy score, ta thấy Model Logistic Regression đạt được độ chính xác cao nhất

# Đánh giá các Model

Tuy nhiên để đánh giá performance của các model thì cách tính bằng model.score không phản ánh chính xác dữ liệu được phân loại như thê nào vì vậy chúng ta cần 1 cách tính độ chính xác khác đó là **F1 Score**

Vì bài toán sarcasm detection thuộc bài toán phân lớp có 2 lớp dữ liệu(sarcasm và not sarcasm) nên ta sử dụng cách đánh giá **True/False Positive/Negative** dựa trên confusion matrix trong thư viện scikit-learn 

**Precision** được định nghĩa là tỉ lệ số điểm true positive trong số những điểm được phân loại là positive

Precision = TP/(TP+FP)
Với : TP(True positive), FP(False positive)

**Recall** được định nghĩa là tỉ lệ số điểm true positive trong số những điểm thực sự là positive

Recall=TP/(TP+FN) Với TP(True positive), FN(False negative)

**F1-score**: trung bình điều hòa giữa **precision** và **recall**

F1-score = 2*(precision*recall)/(precision+recall)

In [20]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

Model Linear support classifier

In [21]:
y_pred=lsvc.predict(features_test)
f1 = f1_score(labels_test, y_pred, average='macro')
print("F1 score: ",f1)

F1 score:  0.842135716647039


Model Gaussian Naive Bayes

In [22]:
y_pred=gnb.predict(features_test)
f1 = f1_score(labels_test, y_pred, average='macro')
print("F1 score: ",f1)

F1 score:  0.7574968610625399


Model Logistic Regression

In [23]:
y_pred=lr.predict(features_test)
f1 = f1_score(labels_test, y_pred, average='macro')
print("F1 score: ",f1)

F1 score:  0.8449540224192296


Model Random Forest Classifier

In [24]:
y_pred=rfc.predict(features_test)
f1 = f1_score(labels_test, y_pred, average='macro')
print("F1 score: ",f1)

F1 score:  0.791225496027812


Sau khi tính được F1 score của từng model, ta thấy F1 score của model Logistic Regression.
- Lý do model Logistic Regression có độ chính xác cao trong các model vì Logistic Regression phù hợp với bài toán classification với 2 class. 
- Các model khác có độ chính xác thấp hơn vì việc xử lý dữ liệu chưa được hoàn chỉnh cũng như độ phù hợp của bài toán.

# Crawl 2000 data mới

Crawl data từ 2 trang web: https://www.theonion.com (tiêu đề châm biếm) và https://www.huffingtonpost.co.uk (tiêu đề không châm biếm) bằng cách sử dụng BeautifulSoup.
- Thư viện Request: dùng đề gửi yêu cầu đến url. 
- Thư viện BeautifulSoup: dùng để lấy dữ liệu từ html(cụ thể ở đây là headline và article_link của từng bài viết).

In [25]:
import requests
from bs4 import BeautifulSoup

Thu thập từ dữ liệu từ hai mục News in brief và News in Photos của trang https://www.theonion.com và đưa dữ liệu vào file dataset.json.

In [26]:
# Đường link đến hai mục của trang Theonion
url = ['https://www.theonion.com/c/news-in-brief', 'https://www.theonion.com/c/news-in-photos']
# Mở file để ghi thêm vào cuối file, nếu không tìm thấy file sẽ tạo mới một file để ghi mới trong drive.
with open('dataset.json', 'a') as file:
    # Vòng for để load hai đường link trong list url
    for i in range(2):
        url_1 = url[i]
        url_2 = url_1
        # Vòng for dùng để chuyển tiếp các trang web để lấy tiêu đề
        for j in range(25):
            # Gửi yêu cầu HTTP bằng request đến url_2.
            r1 = requests.get(url_2)
            # Truy cập r1 dưới dạng nhị phân.
            coverpage = r1.content
            # Thêm parser để phân tích HTML.
            soup2 = BeautifulSoup(coverpage, 'html5lib')
            # Phân tich HTML và tìm tất cả thẻ a, class_='sc-1out364-0 hMndXN js_link', đồng thời truy xuất đến text và href bên trong thẻ.
            coverpage_news_1 = soup2.find_all('a', class_='sc-1out364-0 hMndXN js_link')
  
            t = 12 
            # Vòng while để lấy tất cả tiêu đề và đường dẫn trong một trang web.
            while(t <=50): 
                # Truy suất đường dẫn bài viết.
                a = coverpage_news_1[t]['href']
                # Truy suất tiêu đề bài viết
                b = coverpage_news_1[t].get_text() 
   
                # Ghi vào file.
                file.write('{"is_sarcastic": 1, "headline": "') 
                file.write(b)
                file.write('", "article_link": "')
                file.write(a)
                file.write('"}\n')

                t += 2
                # Cập nhật trang web tiếp theo từ <a class="sc-1out364-0 hMndXN js_link" href="?startTime=1592485200501"> để tiếp tục lấy tiêu đề.
                url_2 = url_1 + coverpage_news_1[51]['href'] 
    file.close()

Thu thập dữ liệu từ hai mục News của trang https://www.huffingtonpost.co.uk và đưa dữ liệu vào file dataset.json

In [27]:
from urllib.request import Request , urlopen 
Data = []
with open('dataset.json', 'a') as file:

  # Biến count dùng để crawl số tiêu đề cần thiết.
  count = 1
  # Vòng for dùng để chuyển tiếp các trang web để lấy tiêu đề
  for i in range(1,34):  

    # Định dạng lại đường đẫn các trang chứa tiêu đề trong phần news tương ướng với i.
    url1 = 'https://www.huffingtonpost.co.uk/news/'+ str(i) +'/?guccounter=2'
    # gửi request HTTP 1.1 đến url với header tự chỉnh 
    req = Request(url1, headers={'User-Agent': 'XYZ/3.0'}) 
    # Mở đường dẫn
    page = urlopen(req,timeout=20) 
    # Thêm parser để phân tích HTML
    soup = BeautifulSoup(page,"html.parser")
     # Phân tich HTML và tìm tất cả thẻ a, class_='sc-1out364-0 hMndXN js_link', đồng thời truy xuất đến text và href bên trong thẻ.
    headlines = soup.find_all('a',attrs={'class':'card__link yr-card-headline'})

    # Vòng lặp dùng để lấy tiêu đề và đường dẫn bài viết.
    for i in range(len(headlines)): 
      # Nếu đủ 1000 tiêu đề thì dùng lại.
      if (count > 1000):
        break
      # Lấy tiêu đề bài viết.
      headline = headlines[i].text
      # Kiểm tra nếu tiêu đề đã được tải thì không tải lại.
      if headline not in Data:
        Data.append(headline)
        # Lấy đường dẫn bài viết.
        link = 'https://www.huffingtonpost.co.uk' + headlines[i]['href']
        # Ghi vào file.
        file.write('{"is_sarcastic": 0, "headline": "') 
        file.write(headline)
        file.write('", "article_link": "')
        file.write(link)
        file.write('"}\n')
        count += 1
  file.close()

# Thực hiện dự đoán trên dataset mới

Đọc dữ liệu của dataset mới

In [28]:
data1 = pd.read_json("dataset.json", lines = True, encoding='utf-8' )
data1.head()

,is_sarcastic,headline,article_link
0,1,Heaven Flush With Cash After Trump Administrat...,https://www.theonion.com/heaven-flush-with-cas...
1,1,Jilted Lover Keying Cheater’s Car Realizes She...,https://www.theonion.com/jilted-lover-keying-c...
2,1,Nancy Pelosi Calls Jamaal Bowman To Scold Him ...,https://politics.theonion.com/nancy-pelosi-cal...
3,1,"Gamers, We Just Spent 4 Days Trapped In A Roll...",https://ogn.theonion.com/gamers-we-just-spent-...
4,1,Defiant Florida Officials Announce They Will I...,https://www.theonion.com/defiant-florida-offic...


Đưa dữ liệu headlines và labels vào các biến tương ứng

In [29]:
features_pred = data1['headline']
labels_pred = data1['is_sarcastic']

# Feature engineering new data

Loại bỏ các ký tự đặc biệt và số trong headlines

In [30]:
# Relacing special symbols and digits in headline column
# re stands for Regular Expression
features_pred = features_pred.apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))

Thực hiện tách từ và loại bỏ các stopwords trong headlines

In [31]:
stop_words = set(stopwords.words('english'))
for i in range (len(features_pred)):
  word_tokens = word_tokenize(features_pred[i])

  filtered_sentence = [w for w in word_tokens if not w in stop_words]

  filtered_sentence = []

  for w in word_tokens:
      if w not in stop_words:
          filtered_sentence.append(w)

Vector hóa dữ liệu văn bản
tv1= tfidVec

In [32]:
tv1 = tfIdfVec
features_pred = tv1.transform(features_pred).toarray()

# Tiến hành dự đoán trên các model

- Thông qua kết quả dự đoán với 2000 headlines mới, ta thấy performance giảm khi sử dụng model được train từ dữ liệu cũ. Lý do performance giảm vì trong dataset mới có các từ chưa từng xuất hiện trong train set và các từ mới xuất hiện hiện nay như(coronavirus,...)



Model Linear support classifier

In [33]:
print("Val accuracy: ",lsvc.score(features_pred, labels_pred))

Val accuracy:  0.772


Model Gaussian Naive Bayes

In [34]:
print("Val accuracy: ",gnb.score(features_pred, labels_pred))

Val accuracy:  0.729


Model Logistic Regression

In [35]:
print("Val accuracy: ",lr.score(features_pred, labels_pred))

Val accuracy:  0.7715


Model Random Forest Classifier

In [36]:
print("Val accuracy: ",rfc.score(features_pred, labels_pred))

Val accuracy:  0.7555


Qua các kết quả dự đoán với dataset mới, ta thấy hiện tượng overfitting xảy ra khi độ chính xác của các model giảm khá nhiều. Phương pháp giải quyết là tiến hành thêm các từ mới xuất hiện.

# Chương trình Sarcasm Detection

Xử lý dữ liệu mới nhập vào

In [37]:
print("Nhap tieu de: ")
ch=(str(input()))
ch = re.sub(r'[^a-zA-Z]',' ',ch)
ch_tokens = word_tokenize(ch)
ch_sentence = [w for w in ch_tokens if not w in stop_words]
ch = " ".join(ch)

Nhap tieu de: 
White House Announces Entire U.S. Populace Of 6,200 Attended Trump’s Tulsa Rally


Đưa dữ liệu về ma trận để dự đoán trên model logistic regression

In [38]:
ch_val = tv1.transform([ch]).toarray()

In [39]:
ch_pred=lr.predict(ch_val)

Sau khi tiến hành dự đoán 10 tiêu đề bất kỳ kết quả dự đoán: 4 đúng -  6 sai, lý do kết quả dự đoán sai vì có nhiều words chưa từng xuất hiện khi được train.

In [40]:
if(ch_pred==1):
  print("sarcasm")
if(ch_pred==0):
  print("not sarcasm")

not sarcasm
